<a href="https://colab.research.google.com/github/keran-w/Projects-in-ML-and-AI/blob/main/ProjML%26AI_hw5_task_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from IPython.display import clear_output
!git clone https://github.com/AntixK/PyTorch-VAE
%cd PyTorch-VAE
!pip install -r requirements.txt
clear_output()

In [ ]:
import torch
from torch import nn

class CVAE(nn.Module):
    """Convolutional variational autoencoder."""

    def __init__(self, latent_dim, num_classes):
    
    
        super(CVAE, self).__init__()
        self.encoder = nn.Sequential(
            nn.LazyConv2d(32, kernel_size=5, padding=2), 
            nn.Relu(),
            nn.LazyConv2d(64, kernel_size=5), 
            nn.Relu(),
            nn.Flatten(1),
            nn.LazyLinear(128), 
            nn.Relu(), # no activation
            nn.LazyLinear(latent_dim + latent_dim)
        )


        self.decoder = nn.Sequential(
            nn.LazyLinear(7*7*32),
            nn.Relu(),
            

            tf.keras.layers.Reshape(target_shape=(7, 7, 32)),
            tf.keras.layers.Conv2DTranspose(
                filters=64, kernel_size=3, strides=2, padding='same',
                activation='relu'),
            tf.keras.layers.Conv2DTranspose(
                filters=32, kernel_size=3, strides=2, padding='same',
                activation='relu'),
            # No activation
            tf.keras.layers.Conv2DTranspose(
                filters=1, kernel_size=3, strides=1, padding='same'),
        )

  @tf.function
  def sample(self, eps=None):
    if eps is None:
      eps = tf.random.normal(shape=(100, self.latent_dim))
    return self.decode(eps, apply_sigmoid=True)

  def encode(self, x):
    mean, logvar = tf.split(self.encoder(x), num_or_size_splits=2, axis=1)
    return mean, logvar

  def reparameterize(self, mean, logvar):
    eps = tf.random.normal(shape=mean.shape)
    return eps * tf.exp(logvar * .5) + mean

  def decode(self, z, apply_sigmoid=False):
    logits = self.decoder(z)
    if apply_sigmoid:
      probs = tf.sigmoid(logits)
      return probs
    return logits